In [2]:
import PyPDF2

#file_path=r"C:\Users\lisa.smith\OneDrive - NHS Supplychain\Desktop\Lisa\04 CEG\commercial-contract-management-all.pdf"

file_path= r"C:\Users\lisa.smith\Documents\Connect\commercial-level-2.pdf"

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            #print(f"Page {page_num + 1}:")
            print(text)

# Example usage
read_pdf(file_path)


Commercial Executive Director
 FTE Total 456.0 (40%)
 Headcount Total 456 (40%)
HayLevel 23
Position Number COMM/CED
Funding SCCL
Director of Diagnostics, Equipment &
Services
 FTE Total 116.0 (10%)
 Headcount Total 116 (10%)
HayLevel 21
Position Number COMM/TD1
Funding SCCLDirector of Medical Technologies
 FTE Total 108.0 (10%)
 Headcount Total 108 (10%)
HayLevel 20
Position Number COMM/TD2
Funding SCCLDirector of Procurement CoE
 FTE Total 23.0 (2%)
 Headcount Total 23 (2%)
HayLevel 21
Position Number COMM/TD3
Funding SCCLDirector of Surgical Consumables
 FTE Total 91.0 (8%)
 Headcount Total 91 (8%)
HayLevel 21
Position Number COMM/TD4
Funding SCCLHead of Commercial & Contract
Management
 FTE Total 21.0 (2%)
 Headcount Total 21 (2%)
HayLevel 20
Funding SCCLHead of Rehab & Community
 FTE Total 48.0 (4%)
 Headcount Total 48 (4%)
HayLevel 20
Position Number COMM/HoC5
Funding SCCLHead of Sourcing Execution
 FTE Total 27.0 (2%)
 Headcount Total 27 (2%)
HayLevel 20
Funding SCCLHead of Supp

In [14]:
import PyPDF2
import pandas as pd

file_path= r"C:\Users\lisa.smith\Documents\Connect\commercial-level-2.pdf"


def read_pdf_into_dataframe(file_path):
    data = []
    current_data = {}
    previous_line = None
    
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            lines = text.split('\n')
            
            for line in lines:
                line = line.strip()
                
                if previous_line is None or previous_line.startswith(('FTE', 'HayLevel')):
                    current_data['Role'] = line
                elif not line.startswith(('FTE', 'HayLevel')):
                    current_data['Role'] += ' ' + line
                elif line.startswith('FTE Total'):
                    current_data['FTE'] = line
                elif line.startswith('Headcount Total'):
                    current_data['Headcount'] = line
                elif line.startswith('HayLevel'):
                    current_data['HayLevel'] = line
                elif line.startswith('Position Number'):
                    current_data['Position Number'] = line
                elif line.startswith('Funding'):
                    current_data['Funding'] = line
                
                previous_line = line
            
            if 'Funding' in current_data:
                data.append(current_data)
                current_data = {}
                previous_line = None
    
    df = pd.DataFrame(data, columns=['Role', 'FTE', 'Headcount', 'HayLevel', 'Position Number', 'Funding'])
    return df

# Example usage
df = read_pdf_into_dataframe(file_path)
df


,Role,FTE,Headcount,HayLevel,Position Number,Funding


In [1]:
import PyPDF2
import re
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import Alignment
from openpyxl.utils import quote_sheetname

def extract_data_from_pdf(filename):
    with open(filename, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        text = ''
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Extract data using regular expressions
    regex_pattern = r'(?P<Role>.+?)\s*(?:FTE Total (?P<FTE>[\d.]+) \(\d+%\))?(?:\s*Headcount Total (?P<Headcount>\d+) \(\d+%\))?\s*' \
                    r'HayLevel (?P<Haylevel>\d+)\s*' \
                    r'Position Number (?P<Position>.+?)\s*' \
                    r'Funding (?P<Funding>.+?)\s*'

    matches = re.findall(regex_pattern, text, re.MULTILINE)

    # Create dataframe
    df = pd.DataFrame(matches, columns=['Role', 'FTE', 'Headcount', 'Haylevel', 'Position', 'Funding'])

    # Clean up the dataframe
   # df['FTE'] = df['FTE'].replace('', pd.NA).astype('float')
    df['Role'] = df['Role'].replace('CCL', '', regex=True)

    return df

# Usage
filename = r"C:\Users\lisa.smith\Documents\Connect\commercial-level-2.pdf"
dataframe = extract_data_from_pdf(filename)

# Write the DataFrame to Excel with freeze panes, filters, and autofit columns
output_path = r"C:\Users\lisa.smith\Documents\Connect\commercial.xlsx"

# Create a workbook and sheet
workbook = Workbook()
sheet = workbook.active

# Add the DataFrame to the sheet
for row in dataframe_to_rows(dataframe, index=False, header=True):
    sheet.append(row)

# Apply filters
sheet.auto_filter.ref = sheet.dimensions

# Autofit column widths
for column in sheet.columns:
    max_length = 0
    column_letter = get_column_letter(column[0].column)
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    sheet.column_dimensions[column_letter].width = adjusted_width

# Freeze panes
sheet.freeze_panes = sheet['A2']

# Save the workbook
workbook.save(output_path)


## Loop through all files

In [ ]:
import os
import PyPDF2
import re
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import Alignment
from openpyxl.utils import quote_sheetname

def extract_data_from_pdf(filename):
    with open(filename, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        text = ''
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    # Extract data using regular expressions
    regex_pattern = r'(?:(?:FTE Total (?P<FTE>[\d.]+) \(\d+%\))|(?:Headcount Total (?P<Headcount>\d+) \(\d+%\)))?\s*' \
                    r'HayLevel (?P<Haylevel>\d+)\s*' \
                    r'Position Number (?P<Position>.+?)\s*' \
                    r'Funding (?P<Funding>.+?)\s*'

    matches = re.findall(regex_pattern, text, re.MULTILINE)

    # Clean up the matches and create dataframe
    cleaned_matches = []
    for match in matches:
        role = match[3].replace('CCL', '')  # Remove 'CCL' from the Role
        cleaned_matches.append((role,) + match[:3] + match[4:])  # Combine the cleaned Role with other details

    # Create dataframe
    df = pd.DataFrame(cleaned_matches, columns=['Role', 'FTE', 'Headcount', 'Haylevel', 'Position', 'Funding'])

    # Clean up the dataframe
    df['FTE'] = df['FTE'].replace('', pd.NA).astype('float')
    df['Headcount'] = df['Headcount'].replace('', pd.NA).astype('Int64')

    return df


# Folder path containing the PDF files
folder_path = r"C:\path\to\folder"

# Loop through PDF files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        # Extract the file name without extension
        output_filename = os.path.splitext(filename)[0]

        # Construct the full file paths
        pdf_file_path = os.path.join(folder_path, filename)
        output_file_path = os.path.join(folder_path, output_filename + ".xlsx")

        # Extract data from PDF and create dataframe
        dataframe = extract_data_from_pdf(pdf_file_path)

        # Write the DataFrame to Excel with freeze panes, filters, and autofit columns
        workbook = Workbook()
        sheet = workbook.active

        # Add the DataFrame to the sheet
        for row in dataframe_to_rows(dataframe, index=False, header=True):
            sheet.append(row)

        # Apply filters
        sheet.auto_filter.ref = sheet.dimensions

        # Autofit column widths
        for column in sheet.columns:
            max_length = 0
            column_letter = get_column_letter(column[0].column)
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            sheet.column_dimensions[column_letter].width = adjusted_width

        # Freeze panes
